# วันที่ 3: การสร้าง Dashboard และวิเคราะห์กรณีศึกษา
## ช่วงบ่าย (13.00 น.–16.00 น.): กรณีศึกษา – วิเคราะห์ข้อมูลจากสถานการณ์จริง

## 3. การวิเคราะห์ข้อมูลเบื้องต้น (ต่อ)

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# กำหนดให้แสดงกราฟในโน้ตบุ๊ก
%matplotlib inline
# กำหนดสไตล์ของกราฟ
plt.style.use('seaborn-v0_8')
# กำหนดให้แสดงคอลัมน์ทั้งหมดในดาต้าเฟรม
pd.set_option('display.max_columns', None)

# โหลดข้อมูลที่ได้สร้างไว้
# หากไม่มีไฟล์ CSV ให้รันโค้ดในไฟล์ 'วันที่3-บ่าย-2-การเตรียมข้อมูล.ipynb' ก่อน

try:
    products_df = pd.read_csv('power_bi_data/products.csv')
    customers_df = pd.read_csv('power_bi_data/customers.csv')
    stores_df = pd.read_csv('power_bi_data/stores.csv')
    sales_df = pd.read_csv('power_bi_data/sales.csv')
    
    # แปลงคอลัมน์ที่เกี่ยวกับวันที่ให้เป็น datetime
    sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
    sales_df['month'] = sales_df['sale_date'].dt.strftime('%Y-%m')
    sales_df['quarter'] = sales_df['sale_date'].dt.to_period('Q').astype(str)
    sales_df['year'] = sales_df['sale_date'].dt.year
    sales_df['weekday'] = sales_df['sale_date'].dt.day_name()
    
    # เชื่อมข้อมูลการขายกับข้อมูลอื่นๆ
    sales_with_info = sales_df.merge(products_df, on='product_id', how='left')
    sales_with_info = sales_with_info.merge(customers_df, on='customer_id', how='left')
    sales_with_info = sales_with_info.merge(stores_df, on='store_id', how='left')
    
    print("โหลดข้อมูลเรียบร้อยแล้ว!")
except FileNotFoundError:
    print("ไม่พบไฟล์ข้อมูล โปรดรันโค้ดในไฟล์ 'วันที่3-บ่าย-2-การเตรียมข้อมูล.ipynb' ก่อน")

### 3.2 การวิเคราะห์ตามภูมิภาคและร้านค้า

In [ ]:
# วิเคราะห์ยอดขายตามภูมิภาค
sales_by_region = sales_with_info.groupby('region').agg({
    'sales_amount': 'sum',
    'profit': 'sum',
    'transaction_id': pd.Series.nunique,
    'customer_id': pd.Series.nunique
}).reset_index()
sales_by_region.columns = ['region', 'sales_amount', 'profit', 'num_transactions', 'num_customers']
sales_by_region['profit_margin'] = (sales_by_region['profit'] / sales_by_region['sales_amount']) * 100
sales_by_region['avg_transaction_value'] = sales_by_region['sales_amount'] / sales_by_region['num_transactions']
sales_by_region = sales_by_region.sort_values('sales_amount', ascending=False)

print("ยอดขายและกำไรตามภูมิภาค:")
display(sales_by_region)

# แสดงกราฟแท่งเปรียบเทียบยอดขายตามภูมิภาค
plt.figure(figsize=(12, 6))
sns.barplot(x='region', y='sales_amount', data=sales_by_region)
plt.title('ยอดขายตามภูมิภาค')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# กราฟวงกลมแสดงสัดส่วนยอดขายตามภูมิภาค
plt.figure(figsize=(10, 8))
plt.pie(sales_by_region['sales_amount'], labels=sales_by_region['region'], autopct='%1.1f%%')
plt.title('สัดส่วนยอดขายตามภูมิภาค')
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# เปรียบเทียบอัตรากำไรตามภูมิภาค
plt.figure(figsize=(12, 6))
sns.barplot(x='region', y='profit_margin', data=sales_by_region)
plt.title('อัตรากำไรตามภูมิภาค')
plt.xticks(rotation=45)
plt.ylabel('อัตรากำไร (%)')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ยอดขายตามจังหวัด (แสดงเฉพาะ 10 อันดับแรก)
sales_by_province = sales_with_info.groupby('province').agg({
    'sales_amount': 'sum',
    'profit': 'sum',
    'transaction_id': pd.Series.nunique
}).reset_index()
sales_by_province['profit_margin'] = (sales_by_province['profit'] / sales_by_province['sales_amount']) * 100
top_provinces = sales_by_province.sort_values('sales_amount', ascending=False).head(10)

print("ยอดขายตามจังหวัด (10 อันดับแรก):")
display(top_provinces)

# แสดงกราฟแท่งเปรียบเทียบยอดขายตามจังหวัด (10 อันดับแรก)
plt.figure(figsize=(12, 6))
sns.barplot(x='province', y='sales_amount', data=top_provinces)
plt.title('ยอดขายตามจังหวัด (10 อันดับแรก)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ประสิทธิภาพร้านค้าตามประเภท
sales_by_store_type = sales_with_info.groupby('store_type').agg({
    'sales_amount': 'sum',
    'profit': 'sum',
    'transaction_id': pd.Series.nunique,
    'store_id': pd.Series.nunique
}).reset_index()
sales_by_store_type.columns = ['store_type', 'sales_amount', 'profit', 'num_transactions', 'num_stores']
sales_by_store_type['profit_margin'] = (sales_by_store_type['profit'] / sales_by_store_type['sales_amount']) * 100
sales_by_store_type['avg_sales_per_store'] = sales_by_store_type['sales_amount'] / sales_by_store_type['num_stores']
sales_by_store_type = sales_by_store_type.sort_values('avg_sales_per_store', ascending=False)

print("ประสิทธิภาพร้านค้าตามประเภท:")
display(sales_by_store_type)

# แสดงกราฟเปรียบเทียบประสิทธิภาพร้านค้าตามประเภท
plt.figure(figsize=(10, 6))
sns.barplot(x='store_type', y='avg_sales_per_store', data=sales_by_store_type)
plt.title('ยอดขายเฉลี่ยต่อร้านตามประเภทร้าน')
plt.tight_layout()
plt.show()

In [ ]:
# เปรียบเทียบอัตรากำไรตามประเภทร้าน
plt.figure(figsize=(10, 6))
sns.barplot(x='store_type', y='profit_margin', data=sales_by_store_type)
plt.title('อัตรากำไรตามประเภทร้าน')
plt.ylabel('อัตรากำไร (%)')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ยอดขายต่อร้านค้าเฉลี่ยตามภูมิภาค
store_performance_by_region = sales_with_info.groupby(['region', 'store_id']).agg({
    'sales_amount': 'sum',
    'profit': 'sum'
}).reset_index()

region_avg_store_performance = store_performance_by_region.groupby('region').agg({
    'store_id': 'count',
    'sales_amount': 'mean',
    'profit': 'mean'
}).reset_index()
region_avg_store_performance.columns = ['region', 'num_stores', 'avg_sales_per_store', 'avg_profit_per_store']
region_avg_store_performance['avg_profit_margin'] = (region_avg_store_performance['avg_profit_per_store'] / region_avg_store_performance['avg_sales_per_store']) * 100
region_avg_store_performance = region_avg_store_performance.sort_values('avg_sales_per_store', ascending=False)

print("ประสิทธิภาพร้านค้าเฉลี่ยตามภูมิภาค:")
display(region_avg_store_performance)

# แสดงกราฟเปรียบเทียบยอดขายเฉลี่ยต่อร้านตามภูมิภาค
plt.figure(figsize=(12, 6))
sns.barplot(x='region', y='avg_sales_per_store', data=region_avg_store_performance)
plt.title('ยอดขายเฉลี่ยต่อร้านตามภูมิภาค')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ร้านค้าที่มีประสิทธิภาพสูงสุด 10 อันดับแรก
top_stores = store_performance_by_region.sort_values('sales_amount', ascending=False).head(10)
top_stores = top_stores.merge(stores_df[['store_id', 'store_name', 'store_type', 'province']], on='store_id')
top_stores['profit_margin'] = (top_stores['profit'] / top_stores['sales_amount']) * 100

print("ร้านค้าที่มีประสิทธิภาพสูงสุด 10 อันดับแรก:")
display(top_stores[['store_name', 'store_type', 'region', 'province', 'sales_amount', 'profit', 'profit_margin']])

# แสดงกราฟแท่งเปรียบเทียบร้านค้าที่มีประสิทธิภาพสูงสุด 10 อันดับแรก
plt.figure(figsize=(12, 6))
sns.barplot(y='store_name', x='sales_amount', data=top_stores)
plt.title('ร้านค้าที่มีประสิทธิภาพสูงสุด 10 อันดับแรก')
plt.tight_layout()
plt.show()

In [ ]:
# วิเคราะห์ความสัมพันธ์ระหว่างอายุร้านค้าและประสิทธิภาพ
# แปลงคอลัมน์วันที่เปิดร้านให้เป็น datetime
stores_df['opening_date'] = pd.to_datetime(stores_df['opening_date'])

# คำนวณอายุร้านค้า (วัน)
current_date = sales_df['sale_date'].max()
stores_df['store_age_days'] = (current_date - stores_df['opening_date']).dt.days

# รวมข้อมูลอายุร้านค้ากับประสิทธิภาพ
store_age_performance = store_performance_by_region.merge(stores_df[['store_id', 'store_age_days', 'store_type']], on='store_id')

# แบ่งกลุ่มอายุร้าน
bins = [0, 365, 2*365, 3*365, 4*365, 5*365]
labels = ['< 1 ปี', '1-2 ปี', '2-3 ปี', '3-4 ปี', '4-5 ปี']
store_age_performance['store_age_group'] = pd.cut(store_age_performance['store_age_days'], bins=bins, labels=labels)

# วิเคราะห์ประสิทธิภาพตามกลุ่มอายุร้าน
age_group_performance = store_age_performance.groupby('store_age_group').agg({
    'store_id': 'count',
    'sales_amount': 'mean',
    'profit': 'mean'
}).reset_index()
age_group_performance.columns = ['store_age_group', 'num_stores', 'avg_sales', 'avg_profit']
age_group_performance['profit_margin'] = (age_group_performance['avg_profit'] / age_group_performance['avg_sales']) * 100

print("ประสิทธิภาพร้านค้าตามกลุ่มอายุร้าน:")
display(age_group_performance)

# แสดงกราฟเปรียบเทียบประสิทธิภาพร้านค้าตามกลุ่มอายุร้าน
plt.figure(figsize=(10, 6))
sns.barplot(x='store_age_group', y='avg_sales', data=age_group_performance)
plt.title('ยอดขายเฉลี่ยตามกลุ่มอายุร้าน')
plt.tight_layout()
plt.show()